# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
The type of problem is Classification as the output labels are "pass" or "fail", which are discrete values.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
from __future__ import print_function #support for python3
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print ("Student data read successfully!")

print (student_data.describe())
display(student_data.head())
print ("Shape:", student_data.shape)

Student data read successfully!
              age        Medu        Fedu  traveltime   studytime    failures  \
count  395.000000  395.000000  395.000000  395.000000  395.000000  395.000000   
mean    16.696203    2.749367    2.521519    1.448101    2.035443    0.334177   
std      1.276043    1.094735    1.088201    0.697505    0.839240    0.743651   
min     15.000000    0.000000    0.000000    1.000000    1.000000    0.000000   
25%     16.000000    2.000000    2.000000    1.000000    1.000000    0.000000   
50%     17.000000    3.000000    2.000000    1.000000    2.000000    0.000000   
75%     18.000000    4.000000    3.000000    2.000000    2.000000    0.000000   
max     22.000000    4.000000    4.000000    4.000000    4.000000    3.000000   

           famrel    freetime       goout        Dalc        Walc      health  \
count  395.000000  395.000000  395.000000  395.000000  395.000000  395.000000   
mean     3.944304    3.235443    3.108861    1.481013    2.291139    3.55443

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


Shape: (395, 31)


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
# Subtract one as last column is target variable
n_features = student_data.shape[1] - 1

#print(student_data.passed.value_counts())

# TODO: Calculate passing students
n_passed = student_data.passed.eq('yes').sum()

# TODO: Calculate failing students
n_failed = student_data.passed.eq('no').sum()

# TODO: Calculate graduation rate
grad_rate = 100*student_data.passed.eq('yes').sum()/n_students

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))



Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print ("\nFeature values:")
print (X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [24]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

ratio_test_size = num_test/X_all.shape[0]

# TODO: Shuffle and split the dataset into the number of training and testing points above
# using stratification as suggested by reviewer, to improve balance of the classes on dataset

X_train, X_test, y_train, y_test = train_test_split(X_all, 
                                                    y_all, 
                                                    test_size=ratio_test_size, 
                                                    random_state=42)


# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

Initially a word about not selected learning models:

Gaussian Naive Bayes (GaussianNB)
 - good for fast classification however generally is not the best score result(less accurate), so will not try.

Decision Trees
 - very good but we will consider in Ensemble methods.

Stochastic Gradient Descent (SGDC)
 - Good for large scale and sparse data, mainly text and NLP tasks. Not our case.

Logistic Regression
 - easy to apply and understand however not usually the best scores.


Selected learning models for exercise:

**Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting), Random Forest for this exercise**

**K-Nearest Neighbors (KNeighbors)**

**Support Vector Machines (SVM)**


 - Detail and answers:

**Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)**

- Real world applications examples:
 - The Application of Ensemble Methods in Forecasting Bankruptcy 
 (This article’s main objective is to compare the predictive power of the two ensemble methods (Adaboost and Bagging) most commonly used in bankruptcy prediction, using a sample consisting of 976 Hungarian corporations.) Ref.: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2756979
 - Ensemble methods for process monitoring in oil and gas industry operations
     In this case emsemble kalman filters for predicting pressure and variables for drilling. Ref.:https://www.sciencedirect.com/science/article/pii/S1875510011000540

- Strengths
 - very tunnable (different learners), parameters of learners
     - for random forests: don´t need scaling of data, parameter tunning to best fit
 - usually state of art performance
- Weakness
 - may not be good for high dimension data

- Good candidate because:
 - We can tune it to improve classification

**K-Nearest Neighbors (KNeighbors)**

- Real world applications examples:
 - Fault Detection Using the k-Nearest Neighbor Rule for Semiconductor Manufacturing Processes
 It has been recognized that effective fault detection techniques can help semiconductor manufacturers reduce scrap, increase equipment uptime, and reduce the usage of test wafers. Using features of process, KNN helped detect failures better and reduce false scraps. Ref.: http://fumblog.um.ac.ir/gallery/170/Fault%20Detection%20Using%20the%20k-Nearest%20Neighbor%20Rule%20for%20Semiconductor%20Manufacturing%20Processes.pdf
 - Automated web usage data mining and recommendation system using K-Nearest Neighbor (KNN) classification method
 Analyse web usage to filter options helping finding the right product or information(categories) on the site. Ref.: http://www.sciencedirect.com/science/article/pii/S221083271400026X

- Strengths
 - Detailed decision boundary
 - fast training ("simply stores instances of the training data")

- Weakness
 - usuallly not so accured
 - less fast to predict

- Good candidate because:
 - small dataset, no problem in performance to predict
 - easy to understand and explain so can be a good **baseline** for compare with more (supposedly) sophisticated methods.
 

**Support Vector Machines (SVM)**

- Real world applications examples:
 - Using support vector machines to forecast the production values of the machinery industry in Taiwan
 SVMs are employed to examine the feasibility in forecasting seasonal time series data of production values of Taiwan’s machinery industry. Ref.: https://link.springer.com/article/10.1007/s00170-004-2139-y
 
 - SVM based defect detection for industrial applications
 Image processing and restoration for identifying the defects in industrial applications. Ref.: http://ieeexplore.ieee.org/document/8014696/?reload=true
 
- Strengths
 - versatile: Possible to tune with hyperparameters (kernels, etc)
 - usually state of art performance
 - efficient in high dimentional spaces (we have 30 original features which was converted to 48 using dummy variables)

- Weakness
 - data scales may impact
 - large datasets are not good (n^3 performance)
 - bad for noise data

- Good candidate because:
 - dataset is not large
 - relative high number of features (48), may impact positively
 

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [25]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [27]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

# TODO: Initialize the three models
clf_A = RandomForestClassifier(random_state = 42)
clf_B = KNeighborsClassifier()
clf_C = svm.SVC(random_state = 42)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
        print('*'*10)
    print ('='*80)

Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0200 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.9844.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7153.
**********
Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0255 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.9964.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7857.
**********
Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0185 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.9951.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.8092.
**********
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8060.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7246.
*

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Ensemble Random Forest **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0200           |    0.0020              |  0.9844          | 0.7153          |
| 200               |        0.0255           |    0.0020              |  0.9964          | 0.7857          |
| 300               |        0.0185           |    0.0020              |  0.9951          | 0.8092          |

** Classifer 2 - KNN **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0010               |  0.0020                |  0.8060          |  0.7246         |
| 200               |    0.0010               |  0.0030                |  0.8800          |  0.7692         |
| 300               |    0.0010               |  0.0030                |  0.8809          |  0.7801         |

** Classifer 3 - SVM  **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0010                |   0.0010               |  0.8777          |  0.7746         |
| 200               |   0.0040                |   0.0020               |  0.8679          |  0.7815         |
| 300               |   0.0090                |   0.0020               |  0.8761          |  0.7838         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

The best model is the Ensemble Random Forest.

It has best f1 score.
It has good prediction time performance, slightly better than SVM (which is very good) and also better than KNN (this is expected as KNN is lazy algorithm). These prediction times varies a lot according to CPU load with other things as this data set is not so large, but these trends happened most of time.

It has the worse training time, but it usually is acceptable.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Imagine that we build all information in a tree, as in your garden. We make a smart division of the data putting related information in nearby tree branches. Each leaf in the branches has a label telling how probable is such a label to happen. If we want to know about this label we go through from trunk to leaf. However, trees can create strange paths to represent irregular data and later this paths may be not so useful for real life labels finding.

If we want to solve this problem we can create a ensemble of smaller trees where we split data and fill them. If we want to know about same information now we look for it in many small trees and average their label values. These trees are smaller and more regular. By going through their branches, we can figure out our information much faster and reliably.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [28]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import ShuffleSplit

# TODO: Create the parameters list you wish to tune
parameters = {
                'max_depth': [None, 1,2,3,4,5,6,7,8,9,10],
                'n_estimators': [5, 10, 50, 100],
                'min_samples_split': [2, 4, 6]
            }

# TODO: Initialize the classifier
clf =  RandomForestClassifier(random_state = 42)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

#Explore all results: do not enable with all parameters set.
#print(grid_obj.cv_results_)

# Get the estimator
clf = grid_obj.best_estimator_

# To explore the selected parameters 
#print(clf.get_params())

# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

Made predictions in 0.0020 seconds.
Tuned model has a training F1 score of 0.8541.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.7755.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **


The tuned parameters selected in GridSearch are:
 - 'max_depth': 4 (different from untuned which is default=None (nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples)
 - 'min_samples_split': 2 (same as untuned, default value)
 - 'n_estimators': 10 (same as untuned, default value)
 - 'min_samples_split': 2 (same as untuned, default value)


The resulting f1-score values are different in tuned model. They are more realistic although slightly worse.

This happens because the GridSearch uses cross-validation (default 'cv=None' uses 3-fold cross validation) and f1-score is averaged from validation sets.


For the same reason the f1-score for training set is much less in GridSearch (0.9951 x 0.8541, for untuned and tuned respectively). The untuned model has overfit for training set.


For testing set results are closer (0.8092 x 0.7755, for untuned and tuned respectively). As commented, the tuned f1-score is more realistic as it is a result of matrix search and cross-validation. 

We expect that the tuned model will provide results more consistently closer to the tuned f1-score (in comparison with untuned f1-score) as we use similar datasets. 



> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.